# Cannon_collect_test

This routine is collecting the data from the training set run

Author(s): Sven Buder

History:
171020: SB Create file

In [1]:
# Compatibility with Python 3
from __future__ import (absolute_import, division, print_function)

try:
    %matplotlib inline
    %config InlineBackend.figure_format='retina'
except:
    pass

# Basic packages
import numpy as np
import os
import sys
import glob
import pickle

# Packages to work with FITS and (IDL) SME.out files
import astropy.io.fits as pyfits
import astropy.table as table
from scipy.io.idl import readsav

# Matplotlib and associated packages for plotting
import matplotlib.pyplot as plt
params = {
    'font.family'        : 'lmodern',
    'font.size'          : 17,
    'axes.labelsize'     : 20,
    'ytick.labelsize'    : 16,
    'xtick.labelsize'    : 16,
    'legend.fontsize'    : 20,
    'text.usetex'        : True, 
    'text.latex.preamble': [r'\usepackage{upgreek}', r'\usepackage{amsmath}'],
    }   
plt.rcParams.update(params)

_parula_data = [[0.2081, 0.1663, 0.5292], 
                [0.2116238095, 0.1897809524, 0.5776761905], 
                [0.212252381, 0.2137714286, 0.6269714286], 
                [0.2081, 0.2386, 0.6770857143], 
                [0.1959047619, 0.2644571429, 0.7279], 
                [0.1707285714, 0.2919380952, 0.779247619], 
                [0.1252714286, 0.3242428571, 0.8302714286], 
                [0.0591333333, 0.3598333333, 0.8683333333], 
                [0.0116952381, 0.3875095238, 0.8819571429], 
                [0.0059571429, 0.4086142857, 0.8828428571], 
                [0.0165142857, 0.4266, 0.8786333333], 
                [0.032852381, 0.4430428571, 0.8719571429], 
                [0.0498142857, 0.4585714286, 0.8640571429], 
                [0.0629333333, 0.4736904762, 0.8554380952], 
                [0.0722666667, 0.4886666667, 0.8467], 
                [0.0779428571, 0.5039857143, 0.8383714286], 
                [0.079347619, 0.5200238095, 0.8311809524], 
                [0.0749428571, 0.5375428571, 0.8262714286], 
                [0.0640571429, 0.5569857143, 0.8239571429], 
                [0.0487714286, 0.5772238095, 0.8228285714], 
                [0.0343428571, 0.5965809524, 0.819852381], 
                [0.0265, 0.6137, 0.8135], 
                [0.0238904762, 0.6286619048, 0.8037619048], 
                [0.0230904762, 0.6417857143, 0.7912666667], 
                [0.0227714286, 0.6534857143, 0.7767571429], 
                [0.0266619048, 0.6641952381, 0.7607190476], 
                [0.0383714286, 0.6742714286, 0.743552381], 
                [0.0589714286, 0.6837571429, 0.7253857143], 
                [0.0843, 0.6928333333, 0.7061666667], 
                [0.1132952381, 0.7015, 0.6858571429], 
                [0.1452714286, 0.7097571429, 0.6646285714], 
                [0.1801333333, 0.7176571429, 0.6424333333], 
                [0.2178285714, 0.7250428571, 0.6192619048], 
                [0.2586428571, 0.7317142857, 0.5954285714], 
                [0.3021714286, 0.7376047619, 0.5711857143], 
                [0.3481666667, 0.7424333333, 0.5472666667], 
                [0.3952571429, 0.7459, 0.5244428571], 
                [0.4420095238, 0.7480809524, 0.5033142857], 
                [0.4871238095, 0.7490619048, 0.4839761905], 
                [0.5300285714, 0.7491142857, 0.4661142857], 
                [0.5708571429, 0.7485190476, 0.4493904762],
                [0.609852381, 0.7473142857, 0.4336857143], 
                [0.6473, 0.7456, 0.4188], 
                [0.6834190476, 0.7434761905, 0.4044333333], 
                [0.7184095238, 0.7411333333, 0.3904761905], 
                [0.7524857143, 0.7384, 0.3768142857], 
                [0.7858428571, 0.7355666667, 0.3632714286], 
                [0.8185047619, 0.7327333333, 0.3497904762], 
                [0.8506571429, 0.7299, 0.3360285714], 
                [0.8824333333, 0.7274333333, 0.3217], 
                [0.9139333333, 0.7257857143, 0.3062761905], 
                [0.9449571429, 0.7261142857, 0.2886428571], 
                [0.9738952381, 0.7313952381, 0.266647619], 
                [0.9937714286, 0.7454571429, 0.240347619], 
                [0.9990428571, 0.7653142857, 0.2164142857], 
                [0.9955333333, 0.7860571429, 0.196652381], 
                [0.988, 0.8066, 0.1793666667], 
                [0.9788571429, 0.8271428571, 0.1633142857], 
                [0.9697, 0.8481380952, 0.147452381], 
                [0.9625857143, 0.8705142857, 0.1309], 
                [0.9588714286, 0.8949, 0.1132428571], 
                [0.9598238095, 0.9218333333, 0.0948380952], 
                [0.9661, 0.9514428571, 0.0755333333], 
                [0.9763, 0.9831, 0.0538]]

from matplotlib.colors import ListedColormap
parula = ListedColormap(_parula_data, name='parula')
parula_zero = _parula_data[0]

# Package to save multiple PDF pages in one PDF
from matplotlib.backends.backend_pdf import PdfPages

In [2]:
# Change Work directory (if Sven's computer)
try:
    localFilePath = '/shared-storage/buder/svn-repos/trunk/GALAH/'
    os.chdir(localFilePath)
    print('Current working directory: '+os.getcwd())
except:
    print('Could not change Path to '+localFilePath)

Current working directory: /shared-storage/buder/svn-repos/trunk/GALAH


In [3]:
tex_dict = dict(
    Teff     = r'$T_\mathrm{eff}~\mathrm{[K]}$',
    Logg     = r'$\log g~\mathrm{[dex]}$',
    Feh      = r'$\mathrm{[Fe/H]~[dex]}$',
    Vmic     = r'$v_\mathrm{mic}~\mathrm{[km/s]}$',
    Vsini    = r'$v_\mathrm{broad}~\mathrm{[km/s]}$',
    Alpha_fe = r'$\mathrm{[\alpha/Fe]~[dex]}$',
    XFe      = r'$\mathrm{[X/Fe]~[dex]}$',
    SNR      = r'$\mathrm{Green~Channel~S/N}$',
    Li        = r'$\mathrm{[Li/Fe]~[dex]}$',
    C        = r'$\mathrm{[C/Fe]~[dex]}$',
    O        = r'$\mathrm{[O/Fe]~[dex]}$',
    Na       = r'$\mathrm{[Na/Fe]~[dex]}$',
    Mg       = r'$\mathrm{[Mg/Fe]~[dex]}$',
    Al       = r'$\mathrm{[Al/Fe]~[dex]}$',
    Si       = r'$\mathrm{[Si/Fe]~[dex]}$',
    K        = r'$\mathrm{[K/Fe]~[dex]}$',
    Ca       = r'$\mathrm{[Ca/Fe]~[dex]}$',
    Sc       = r'$\mathrm{[Sc/Fe]~[dex]}$',
    Ti       = r'$\mathrm{[Ti/Fe]~[dex]}$',
    V        = r'$\mathrm{[V/Fe]~[dex]}$',
    Cr       = r'$\mathrm{[Cr/Fe]~[dex]}$',
    Mn       = r'$\mathrm{[Mn/Fe]~[dex]}$',
    Co       = r'$\mathrm{[Co/Fe]~[dex]}$',
    Ni       = r'$\mathrm{[Ni/Fe]~[dex]}$',
    Zn       = r'$\mathrm{[Zn/Fe]~[dex]}$',
    Rb       = r'$\mathrm{[Rb/Fe]~[dex]}$',
    Sr       = r'$\mathrm{[Sr/Fe]~[dex]}$',
    Y        = r'$\mathrm{[Y/Fe]~[dex]}$',
    Zr       = r'$\mathrm{[Zr/Fe]~[dex]}$',
    Mo       = r'$\mathrm{[Mo/Fe]~[dex]}$',
    Ru       = r'$\mathrm{[Ru/Fe]~[dex]}$',
    Ba       = r'$\mathrm{[Ba/Fe]~[dex]}$',
    La       = r'$\mathrm{[La/Fe]~[dex]}$',
    Ce       = r'$\mathrm{[Ce/Fe]~[dex]}$',
    Nd       = r'$\mathrm{[Nd/Fe]~[dex]}$',
    Sm       = r'$\mathrm{[Sm/Fe]~[dex]}$',
    Eu       = r'$\mathrm{[Eu/Fe]~[dex]}$'
    )

In [4]:
def export_fits(dictionary, filename):
    """
    This is a function to export a datastructure, nomatter where you currently are
    
    INPUT:
    dictionary : dictionary data_structure
    filename   : how should the file be called? Best: include the full path
    
    OUTPUT:
    FITS file saved as filename
    """
    
    t = table.Table()
    for each_key in dictionary.keys():
        t.add_column(table.Column(name=each_key, data=dictionary[each_key]))
    t.write(filename+'.fits',overwrite=True)

In [5]:
def create_galah_dict(wg3_wg4_setup):
    galah_dict = dict(

        # General
        sobject_id     = 112233000001234,
        galah_id       = 1234567,
        field_id       = 0,
        ra             = 0.0,
        dec            = 0.0,
        ebv            = 0.0,
        snr_c1_iraf    = 0.0,
        snr_c2_iraf    = 0.0,
        snr_c3_iraf    = 0.0,
        snr_c4_iraf    = 0.0,
        red_flag       = 0,
        
        # GUESS
        teff_guess     = 0.0,
        logg_guess     = 0.0,
        feh_guess      = 0.0,
        rv_guess       = 0.0,
        e_rv_guess     = 0.0,
        rv_guess_shift = 0.0,
        flag_guess     = 0,

    #     # General SME
    #     trained_on     = 0,
    #     rv_sme         = 0.0,
    #     e_rv_sme       = 0.0,
    #     Alpha_fe_sme   = 0.0,
    #     e_Alpha_fe_sme = 0.0,

        # General Cannon
        chi2_cannon       = 0.0,
        sp_label_distance = 0.0,
        flag_cannon       = 0,
        Alpha_fe_cannon   = 0.0,
        e_Alpha_fe_cannon = 0.0
        )

    #for each_method in ['sme','cannon']:
    for each_method in ['cannon']:
        for each_sp in wg3_wg4_setup['stellar_parameters']:
            galah_dict[each_sp+'_'+each_method] = 0.0
            galah_dict['e_'+each_sp+'_'+each_method] = 0.0

        for each_elem in wg3_wg4_setup['elements']:
            galah_dict[each_elem+'_abund_'+each_method]          = 0.0
            galah_dict['e_'+each_elem+'_abund_'+each_method]     = 0.0
            galah_dict['ld_'+each_elem+'_abund_'+each_method]    = 0.0
            galah_dict['flag_'+each_elem+'_abund_'+each_method]  = 0
            galah_dict['depth_'+each_elem+'_abund_'+each_method] = 0.0
            galah_dict['sn_'+each_elem+'_abund_'+each_method]    = 0.0
            galah_dict['chi2_'+each_elem+'_abund_'+each_method]  = 0.0
            
    return galah_dict

In [6]:
def read_model_pickle(filename):
    """
    INPUT:
    
    filename : pickle filename path
    
    OUTPUT:
    pickle file structure
    
    """
    door = open(filename,'r')
    print('Will read pickle file: '+filename)
    model_data = pickle.load(door)
    return model_data

In [7]:
def plot_HRD(dictionary, sme_cannon='cannon', savefig=False, xlim=(7900,3600), ylim=(5.,0.)):
    HRD_kwargs = dict(cmap=parula, s=20, lw=0.1, rasterized=True)
    f, (ax1,ax2) = plt.subplots(1,2,figsize=(2*6.4, 4.8))

    s1 = ax1.scatter(dictionary['Teff_'+sme_cannon],dictionary['Logg_'+sme_cannon],c=dictionary['Feh_'+sme_cannon], vmin=-2.5, vmax=0.5,**HRD_kwargs)
    ax1.set_xlabel(tex_dict['Teff'])
    ax1.set_ylabel(tex_dict['Logg'])
    c1 = plt.colorbar(s1, ax=ax1)
    c1.set_label(tex_dict['Feh'])
    ax1.set_xlim(xlim)
    ax1.set_ylim(ylim)
    
    s2 = ax2.scatter(dictionary['Teff_'+sme_cannon],dictionary['Logg_'+sme_cannon],c=dictionary['snr_c2_iraf'], vmin=25, vmax=160,**HRD_kwargs)
    ax2.set_xlabel(tex_dict['Teff'])
    ax2.set_ylabel(tex_dict['Logg'])
    c2 = plt.colorbar(s2, ax=ax2)
    c2.set_label(tex_dict['SNR'])
    ax2.set_xlim(xlim)
    ax2.set_ylim(ylim)

    plt.tight_layout()
    if savefig!=False:
        plt.savefig(savefig+'.pdf')

In [8]:
def ab_scatter(X, Y, ax=plt.gca, **kwargs):
    """
    This function gives back a scatter plot
    
    """

    c = kwargs.get('c',parula_zero)
    s = kwargs.get('s',2)
    s1 = ax.scatter(X,Y,c=c,s=s,alpha=0.5,rasterized=True)
    
    return ax

def ab_dens2d(X, Y, ax=plt.gca, min_per_bin=5, zeroslines=True, interimlines=True, colorbar=True, **kwargs):
    """
    This function gives back a 2D density plot 
    of the data put in as X and Y with 
    all points as scatter below certain density
    
    """
    
    #first make sure to only use finite X and Y values
    XY_finite = (np.isfinite(X) & np.isfinite(Y))
    X = X[XY_finite]
    Y = Y[XY_finite]
    
    # General kwargs:
    xlabel = kwargs.get('xlabel','')
    ylabel = kwargs.get('ylabel', r'$\mathrm{[X/Fes]~[dex]}$')
    xlim   = kwargs.get('xlim', (-3.0,0.65))
    ylim   = kwargs.get('ylim', (-0.5,1.00))
    cmap = kwargs.get('cmap', parula)
    bins = kwargs.get('bins', (0.05,0.025))
    if np.shape(bins) != ():
        # assuming input in dex
        bins = [np.arange(xlim[0],xlim[1],bins[0]),np.arange(ylim[0],ylim[1],bins[1])]
    
    # plot all points as scatter before density structure is overlaid
    scatter = ab_scatter(X,Y,ax=ax)

    H, xedges, yedges = np.histogram2d(X,Y,bins=bins)
    H=np.rot90(H)
    H=np.flipud(H)
    Hmasked = np.ma.masked_where(H<min_per_bin,H)

    dens2d=ax.pcolormesh(xedges,yedges,Hmasked,cmap=cmap)

    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)
    xticks = kwargs.get('xticks',ax.get_xticks())
    ax.set_xticks(xticks)
    
    if zeroslines == True:
        ax.axhline(0,c='k',lw=0.5)
        ax.axvline(0,c='k',lw=0.5)
        
    if interimlines == True:
        ax.axhline(0.1,c='k',linestyle='--')
        ax.axhline(0.2,c='k',linestyle='--')
        ax.axhline(0.3,c='k',linestyle='--')
        ax.axhline(0.4,c='k',linestyle='--')
        ax.axhline(0.5,c='k',linestyle='--')
        ax.axhline(-0.1,c='k',linestyle='--')
        ax.axhline(-0.2,c='k',linestyle='--')

    if colorbar == True:
        c = plt.colorbar(dens2d,ax=ax)
        c.set_label('Counts')

In [9]:
def plot_alpha(dictionary, sme_cannon='cannon', savefig=False, xlim=(-3.0,0.65), ylim=(-0.5,1.00)):
    
    interimlines = False#True

    plt.figure(figsize=(15,5))
    for each,mode in enumerate(['Alpha_fe','O','Mg','Si','Ca','Ti']):
        ax = plt.subplot(2,3,each+1)
        if each == 0:
            X = dictionary['Feh_'+sme_cannon]
            Y = dictionary['Alpha_fe_'+sme_cannon]
        else:
            good = dictionary['flag_'+mode+'_abund_'+sme_cannon] == 0
            X = dictionary['Feh_'+sme_cannon][good]
            Y = dictionary[mode+'_abund_'+sme_cannon][good]
        ab_dens2d(
            ax=ax,
            xlim=xlim,
            ylim=ylim,
            bins=(0.1,0.05),
            X = X,
            Y = Y,
            xlabel='',
            ylabel='',
            interimlines=interimlines
            );
        ax.text(0.05,0.1,tex_dict[mode],transform=ax.transAxes)
        if each in [0,3]:
            ax.set_ylabel(tex_dict['XFe'])
        if each < 3:
            ax.set_xticks([])
        if each > 2:
            ax.set_xlabel(tex_dict['Feh'])
        ax.set_xticks(ax.get_xticks()[::2])
        ax.set_yticks(ax.get_yticks()[::2])
    plt.tight_layout()

    if savefig!=False:
        plt.savefig(savefig+'.pdf')

In [10]:
def get_detections(self, mode='Li'):
    """
    This algorithm helps us to decide if an element abundance measurement is
    
    significant  (flag==0)
    upper limit  (flag==1)
    bad chi2 fit (flag+2)
    
    INPUT:
    mode : element for which the detections and flags have to be estimated
    
    """
    
    # First initialise flags as -1
    self.cannon['flag_'+mode+'_abund_cannon'][:] = 4
    
    # Now we start the flagging for detection/upper limit/bad chi2

    # first get the flux/sn within line mask (mob==1 analogous to SME)
    mob1    = np.array(self.masks[mode],dtype=bool)
    sn_mob1 = np.array(map(lambda x: np.mean(self.spectrum_flux[mode][x,mob1]/self.spectrum_error[mode][x,mob1]), range(len(self.cannon['sobject_id']))))
    sn_mob1[sn_mob1 < 1.] = 1.
    sn_mob1[np.isnan(sn_mob1)] = 1.
    chi2_mob1 = np.array(map(lambda x: np.mean(self.model_chi2[mode][x,mob1]), range(len(self.cannon['sobject_id']))))
    
    # Upper limit criterium is 2-sigma and minimum line depth of 0.05
    upper_limit        = 2./sn_mob1; upper_limit[upper_limit < 0.05] = 0.05
    # Upper limit criterium is 3-sigma and minimum line depth of 0.05
    significance_limit = 3./sn_mob1; significance_limit[significance_limit < 0.05] = 0.05
    
    # Line depth within segment, assuming continuum==1.
    line_depth = 1. - np.array(map(lambda x: self.spectrum_flux[mode][x,mob1], range(len(self.cannon['sobject_id']))))

    self.cannon['depth_'+mode+'_abund_cannon'] = np.array([np.max(line_depth[x]) for x in range(len(line_depth))])
    
    # Now for each pixel within line mask check if line depth larger than upper limit / significance limit
    upper_limit_check        = np.array([line_depth[x,:] >= upper_limit[x] for x in range(len(line_depth[:,0]))])
    significance_limit_check = np.array([line_depth[x,:] >= significance_limit[x] for x in range(len(line_depth[:,0]))])

    # For upper limit, set flag to 1
    self.cannon['flag_'+mode+'_abund_cannon'][np.array([np.any(upper_limit_check[x]) for x in range(len(self.cannon['sobject_id']))])] = 1
    # For significant detection, set flag to 0
    self.cannon['flag_'+mode+'_abund_cannon'][np.array([np.any(significance_limit_check[x]) for x in range(len(self.cannon['sobject_id']))])] = 0

    # Set flag, if chi2 within line mask is above typical value for given S/N within line mask
    self.cannon['sn_'+mode+'_abund_cannon']   = sn_mob1
    self.cannon['chi2_'+mode+'_abund_cannon'] = chi2_mob1
    
    # Here we should adjust the S/N depending function
    def chi2_limit(log_sn = np.log10(sn_mob1)):
        return 4. + 2. * log_sn

    #plt.plot(np.log10(sn_mob1), np.log10(chi2_mob1), 'ko')
    #plt.plot(np.arange(0.0,3.0,0.1), chi2_limit(np.arange(0.0,3.0,0.1)), 'r')
        
    chi2_high = (chi2_mob1 > chi2_limit(log_sn = np.log10(sn_mob1)))
    self.cannon['flag_'+mode+'_abund_cannon'][chi2_high] += 2

In [11]:
def label_distance(self, mode='Sp'):
    """
    Here we flag those solutions far away from the Training set
    (similar to LAMOST's D from Ho et al. 2016)
    """

    precision = np.load(localFilePath+'CANNON/'+self.reduction_DR+'/'+self.version_cannon+'/fits_files/'+self.version_cannon+'_'+self.setup_cannon+'_'+self.reduction_DR+'_'+str(self.nr_ccds_cannon)+'ccds_precision.npy').item()

    t = table.Table()
    ts = t.read(localFilePath+'CANNON/'+self.reduction_DR+'/'+self.version_cannon+'/trainingset/'+self.version_cannon+'_'+mode+'_'+self.setup_cannon+'_trainingset.fits')

    if mode == 'Sp':
        D_part = np.zeros(
            (len(ts['sobject_id']), len(self.cannon['sobject_id'])),
            dtype = ([('Teff',float),('Logg',float),('Feh',float),('Vsini',float)]))
    else:
        D_part = np.zeros(
            (len(ts['sobject_id']), len(self.cannon['sobject_id'])),
            dtype = ([('Teff',float),('Logg',float),('Feh',float),('Vsini',float),(mode,float)]))

    D_part['Teff']  = map(lambda x: 1/(precision['cannon_rms']['Teff'] )**2. * (self.cannon['Teff_cannon'] - ts['Teff_sme'][x])**2.,range(len(ts['sobject_id'])))
    D_part['Logg']  = map(lambda x: 1/(precision['cannon_rms']['Logg'] )**2. * (self.cannon['Logg_cannon'] - ts['Logg_sme'][x])**2.,range(len(ts['sobject_id'])))
    D_part['Feh']   = map(lambda x: 1/(precision['cannon_rms']['Feh']  )**2. * (self.cannon['Feh_cannon'] - ts['Feh_sme'][x])**2.,range(len(ts['sobject_id'])))
    D_part['Vsini'] = map(lambda x: 1/(precision['cannon_rms']['Vsini'])**2. * (self.cannon['Vsini_cannon'] - ts['Vsini_sme'][x])**2.,range(len(ts['sobject_id'])))
    if mode == 'Sp':
        D_sum = D_part['Teff'] + D_part['Logg'] + D_part['Feh'] + D_part['Vsini']
    else:
        D_part[mode] = map(lambda x: 1/(precision['cannon_rms'][mode] )**2. * (self.cannon[mode+'_abund_cannon'] - ts[mode+'_abund_sme'][x])**2.,range(len(ts['sobject_id'])))
        D_sum = D_part['Teff'] + D_part['Logg'] + D_part['Feh'] + D_part['Vsini'] + D_part[mode]

    # We follow Ho et al. 2016 by calculating the mean of the 10 smallest label-distances
    D = np.array(map(lambda x: np.mean(D_sum[:,x][D_sum[:,x].argsort()[:10]]),range(len(self.cannon['sobject_id']))))
    plt.figure()
    plt.hist(D[np.isfinite(D)],bins=np.arange(0,21,0.25));
    plt.xlabel('D ('+mode+')')
    plt.savefig(localFilePath+'CANNON/'+self.reduction_DR+'/'+self.version_cannon+'/diagnostic_plots/Label_distances_'+str(self.obs_date)+'_'+mode+'.png',dpi=300)
    plt.close()

    sp_label_distance_limit = 2*4. # 2 sigma for 4 freedoms
    ab_label_distance_limit = 2*5. # 2 sigma for 5 freedoms
    if mode=='Sp':
        self.cannon['sp_label_distance'] = D
        self.cannon['flag_cannon'][D > sp_label_distance_limit] += 1
    else:
        self.cannon['ld_'+mode+'_abund_cannon'] = D
        self.cannon['flag_'+mode+'_abund_cannon'][D > ab_label_distance_limit] += 1
        self.cannon['flag_'+mode+'_abund_cannon'][self.cannon['sp_label_distance'] > sp_label_distance_limit] += 2

In [12]:
def compute_alpha_fe(data_structure,sme_cannon='cannon',alpha_elements=['O','Mg','Si','Ca','Ti']):
    """
    INPUT:
    
    data_structure : Structure, from which the alpha-process elements come from
                     This structure also has to have the keys 
                     'Alpha_fe_'+sme_cannon and 'e_Alpha_fe_'+sme_cannon
    sme_cannon     : 'sme' or 'cannon' which will be use as key within data_structure
                     default: 'cannon'
    alpha_elements : the array including which alpha-process elements shall be included
                     default: ['O','Mg','Si','Ca','Ti']
    
    OUTPUT:
    
    None, but the following entries will be filled:
    data_structure['Alpha_fe_'+sme_cannon] & data_structure['e_Alpha_fe_'+sme_cannon] 
    
    """
    for each_sobject_id in range(len(data_structure['sobject_id'])):

        combi = np.nansum(np.concatenate([(data_structure[x+'_abund_'+sme_cannon][each_sobject_id]/data_structure['e_'+x+'_abund_'+sme_cannon][each_sobject_id]**2.)[(np.isfinite(data_structure[x+'_abund_'+sme_cannon][each_sobject_id]) & (data_structure['flag_'+x+'_abund_'+sme_cannon][each_sobject_id] == 0))] for x in alpha_elements]))/np.nansum(np.concatenate([1./data_structure['e_'+x+'_abund_'+sme_cannon][each_sobject_id][np.isfinite(data_structure[x+'_abund_'+sme_cannon][each_sobject_id]) & (data_structure['flag_'+x+'_abund_'+sme_cannon][each_sobject_id] == 0)]**2. for x in alpha_elements]))
        e_combi = np.concatenate([1./(data_structure['e_'+x+'_abund_'+sme_cannon][each_sobject_id])[np.isfinite(data_structure[x+'_abund_'+sme_cannon][each_sobject_id]) & (data_structure['flag_'+x+'_abund_'+sme_cannon][each_sobject_id] == 0)]**2. for x in alpha_elements])
        # If there are enough unflagged values to compute Alpha_fe, go ahead
        if len(e_combi) > 0:
            e_combi = np.sqrt(1./np.nansum(e_combi))
            data_structure['Alpha_fe_'+sme_cannon][each_sobject_id] = combi
            data_structure['e_Alpha_fe_'+sme_cannon][each_sobject_id] = e_combi
        # Otherwise set values to np.nan
        else:
            data_structure['Alpha_fe_'+sme_cannon][each_sobject_id] = np.nan
            data_structure['e_Alpha_fe_'+sme_cannon][each_sobject_id] = np.nan

## The class 'testset', where we want to put in all important data and routines

In [13]:
class testset(object):
    """ 
    Trainingset is the major class for the important information on the trainingset.
    
    In the future, this should be executable without the trainingset class
    
    Initialise with:
    obs_date      : Date of observation for the particular testset
    wg3_wg4_setup : setup dictionary with all relevant data for IRAF/SME/CANNON
    
    """
    
    def __init__(self, obs_date, wg3_wg4_setup):
        os.chdir(localFilePath)
        self.obs_date           = obs_date
        if not hasattr(self,'galah_dict'):
            self.galah_dict = create_galah_dict(wg3_wg4_setup)
        self.version_cannon     = wg3_wg4_setup['version_cannon']
        self.setup_cannon       = wg3_wg4_setup['setup_cannon']
        self.nr_ccds_cannon     = wg3_wg4_setup['nr_ccds_cannon']
        self.version_reduction  = wg3_wg4_setup['version_reduction']
        self.reduction_DR       = wg3_wg4_setup['reduction_DR']
        self.version_sme        = wg3_wg4_setup['version_sme']
        self.setup_sme          = wg3_wg4_setup['setup_sme']
        self.stellar_parameters = wg3_wg4_setup['stellar_parameters']
        self.elements           = wg3_wg4_setup['elements']

        os.chdir('CANNON/'+self.reduction_DR+'/'+self.version_cannon)
        print('You are now working in:'+os.getcwd())
        self.SP_data = 'pickle_test/'+self.version_cannon+'_Sp_'+self.setup_cannon+'_'+self.reduction_DR+'_'+str(self.nr_ccds_cannon)+'ccds_'+str(self.obs_date)+'_tags.pickle'
        print(self.SP_data)

        self.AB_data  = {}
        if self.elements!='None':
            for each_element in self.elements:
                try:
                    found_setup = glob.glob('pickle_test/'+self.version_cannon+'_'+each_element+'_'+self.setup_cannon+'_'+self.reduction_DR+'_'+str(self.nr_ccds_cannon)+'ccds_'+str(self.obs_date)+'_tags.pickle')
                    self.AB_data[each_element] = found_setup[-1]
                    print('Found automatic setup configuration for '+each_element+': '+found_setup[-1])
                except:
                    print('There is no setup configuration for '+each_element)
        self.cannon_trainingset_filename = 'sobject_iraf_'+self.version_cannon+'_trainingset.fits'
        self.cannon_filename = 'sobject_iraf_'+self.version_cannon+'.fits'

        self.wave               = {}
        self.wave['ccd1']       = np.arange(4715.94,4896.00,0.046) # ab lines 4716.3 - 4892.3
        self.wave['ccd2']       = np.arange(5650.06,5868.25,0.055) # ab lines 5646.0 - 5867.8
        self.wave['ccd3']       = np.arange(6480.52,6733.92,0.064) # ab lines 6481.6 - 6733.4
        self.wave['ccd4']       = np.arange(7693.50,7875.55,0.074) # ab lines 7691.2 - 7838.5
        self.wave['all']        = np.concatenate((self.wave['ccd1'],self.wave['ccd2'],self.wave['ccd3'],self.wave['ccd4']))
        
        self.model_flux         = {}
        self.model_chi2         = {}
        self.spectrum_flux      = {}
        self.spectrum_error     = {}
        self.masks              = {}

    def initiate_cannon_structure(self,nstars):
        if not hasattr(self,'cannon'):
            print('cannon structure does not exist yet - initialising cannon structure now')
            self.cannon = {}
            for each_key in self.galah_dict.keys():
                self.cannon[each_key] = np.array([self.galah_dict[each_key] for x in range(nstars)])
        else: 
            print('You want to initialise cannon structure, but it already exists!')
        
    def get_SME(self):
        found_SME = glob.glob('trainingset/'+self.version_cannon+'_Sp_*_trainingset.fits')
        print('Found automatic setup configuration for SME training set:')
        print(found_SME[-1])
        self.SME=pyfits.getdata(found_SME[-1])

    def get_SP(self):
        """
        INPUT:
        None

        OUTPUT:
        Adds all important information from the stellar parameter run of the Cannon to the class
        """
        params, e_params, covs, chi2, sobject_id, chi2good, chi2each = read_model_pickle(self.SP_data)

        if not hasattr(self,'cannon'):
            self.initiate_cannon_structure(len(sobject_id))

        self.cannon['sobject_id'] = np.array([int(sobject_id[x]) for x in range(len(sobject_id))])

        for index, each_param in enumerate(self.stellar_parameters):
            self.cannon[each_param+'_cannon'] = params[:,index]
            self.cannon['e_'+each_param+'_cannon'] = e_params[:,index]
        self.cannon['chi2_cannon'] = chi2good

        # Neglect those with chi2 above high and below low limit (expectation is >1.0)
        # Chosen because of 5 times too high chi2
        high_chi2_limit = 5.
        # Chosen because of strange overdensity below 0.4
        low_chi2_limit = 0.4 
        self.cannon['flag_cannon'][self.cannon['chi2_cannon'] > high_chi2_limit] += 2 # chi2 too high
        self.cannon['flag_cannon'][self.cannon['chi2_cannon'] <  low_chi2_limit] += 2 # chi2 too low


        #     red_flag
        #   0 for no flags
        # + 1 for bad wavelength solution in ccd_1, 
        # + 2 for bad wavelength solution in ccd_2, 
        # + 4 for bad wavelength solution in ccd_3, 
        # + 8 for bad wavelength solution in ccd_4,
        # +16 for molecfit fail in ccd_3
        # +32 for molecfit fail in ccd_4
        # +64 for twilight flat

        # Reduction fail
        self.cannon['flag_cannon'][self.cannon['red_flag'] > 0] += 4

        # Estimate label distance for stellar parameters
        label_distance(self, mode='Sp')  

    def get_IRAF(self):
        if hasattr(self,'cannon'):
            print('Reading in IRAF reduction version: '+self.version_reduction)
            iraf = pyfits.getdata(localFilePath+'/DATA/'+self.version_reduction+'.fits',ext=1)
            for cannon_index,each_sobject_id in enumerate(self.cannon['sobject_id']):
                iraf_equivalent = np.where(each_sobject_id == iraf['sobject_id'])[0]
                if len(iraf_equivalent) > 0:
                    for each_iraf_label in [
                            'galah_id','field_id','ra','dec','ebv',
                            'snr_c1_iraf','snr_c2_iraf','snr_c3_iraf','snr_c4_iraf','red_flag',
                            'teff_guess','logg_guess','feh_guess','rv_guess','e_rv_guess','rv_guess_shift','flag_guess'
                            ]:
                        self.cannon[each_iraf_label][cannon_index] = iraf[each_iraf_label][iraf_equivalent[0]]
                else:
                    print('Something went wrong here, because there is no IRAF entry for: '+str(each_sobject_id))

            # Once snr_c2_iraf is filled, we can update the stellar parameter errors
            precision = np.load(localFilePath+'CANNON/'+self.reduction_DR+'/'+self.version_cannon+'/fits_files/'+self.version_cannon+'_'+self.setup_cannon+'_'+self.reduction_DR+'_'+str(self.nr_ccds_cannon)+'ccds_precision.npy').item()
            def precision_snr(x, floor, amp, damp):
                return abs(floor) + amp * np.exp( - x *damp)
            for each_label in self.stellar_parameters:
                self.cannon['e_'+each_label+'_cannon'] = (
                        (self.cannon['e_'+each_label+'_cannon'])**2. + 
                        (precision_snr(self.cannon['snr_c2_iraf'], *precision['cannon_precision_function'][each_label]))**2.
                    )**0.5

            self.cannon['flag_cannon'][(self.cannon['ra'] == 0) & (self.cannon['dec'] == 90)] += 2 # deselected special pointings

        else:
            print('You did not call testset.get_SP() yet, which initialises the CANNON attribute to save the abundances in!')

    def get_tSNE(self):

        t = table.Table()
        tsne = t.read(localFilePath+'DATA/dr52_class_joined.fits')

        """
        Individual classifications are:

        The ones we will use:

        binary
        triple

        HaHb emission

        problematic
        problematic - ccd2 continuum
        problematic - ccd3 continuum

        problematic - osc. cont.
        problematic - ccd4
        problematic - ccd4 oversubtraction
        problematic - neg. flux
        problematic - negative flux
        problematic - ccd1

        problematic - ccd3 spikes
        problematic - ccd4 spikes
        problematic - ccd4 strong spike

        The ones we wont use

        mol. abs. bands
        hot stars
        CMP giants

        """

        tsne_flag = {}

        tsne_flag['binary'] = 8
        tsne_flag['trippe'] = 8

        tsne_flag['problematic - neg. flux'] = 16
        tsne_flag['problematic - negative flux'] = 16

        tsne_flag['problematic - osc. cont.'] = 32
        tsne_flag['problematic - ccd2 continuum'] = 32
        tsne_flag['problematic - ccd3 continuum'] = 32

        tsne_flag['problematic'] = 32
        tsne_flag['problematic - ccd1'] = 32
        tsne_flag['problematic - ccd3 spikes'] = 32

        tsne_flag['problematic - ccd4'] = 64
        tsne_flag['problematic - ccd4 oversubtraction'] = 64
        tsne_flag['problematic - ccd4 spikes'] = 64
        tsne_flag['problematic - ccd4 strong spike'] = 64

        tsne_flag['HaHb emission'] = 128

        for each_tsne_flag in tsne_flag.keys():

            run1 = tsne['sobject_id'][tsne['class_norm']     ==each_tsne_flag]
            run2 = tsne['sobject_id'][tsne['noIR_class_norm']==each_tsne_flag]

            exclude_tsne_flagged = []
            if (len(run1) > 0) & (len(run2) > 0):
                exclude_tsne_flagged = np.unique(
                                       np.concatenate((
                                           tsne['sobject_id'][tsne['class_norm']     ==each_tsne_flag],
                                           tsne['sobject_id'][tsne['noIR_class_norm']==each_tsne_flag]
                                           ))
                                       )
            elif len(run1) > 0:
                exclude_tsne_flagged = run1
            elif len(run2) > 0:
                exclude_tsne_flagged = run2

            for each_sobject_id_to_glag in exclude_tsne_flagged:
                found = np.where(each_sobject_id_to_glag == self.cannon['sobject_id'])[0]
                if len(found) > 0:
                    self.cannon['flag_cannon'][found[0]] += tsne_flag[each_tsne_flag]
                    
    def get_AB(self):
        """
        Routine to get all abundance information

        INPUT:
        None, but class attribute 'cannon' must exist!

        OUTPUT:
        Adds all important information from the element abundance run of the Cannon to the class
        """
        if hasattr(self,'cannon'):
            for each_element in self.elements:
                try:

                    # Get line masks
                    self.masks[each_element] = np.loadtxt(localFilePath+'CANNON/'+self.reduction_DR+'/masks_4ccds/'+self.reduction_DR+'_'+each_element+'.txt',usecols=(5,),unpack=1)

                    # Get labels from Cannon
                    params, e_params, covs, chi2, sobject_ids, chi2good, chi2each = read_model_pickle(self.AB_data[each_element])
                    self.cannon[each_element+'_abund_cannon'] = params[:,-1]

                    # Read in test set spectral data
                    testdataname=localFilePath+'CANNON/'+self.reduction_DR+'/pickle_'+self.reduction_DR+'_4ccds/'+self.reduction_DR+'_4ccds_'+str(self.obs_date)+'.pickle'
                    door=open(testdataname,'r')
                    testdataall, ids, galah_name = pickle.load(door)
                    door.close()

                    # Read in model for element
                    use_model = open(localFilePath+'CANNON/'+self.reduction_DR+'/'+self.version_cannon+'/'+self.version_cannon+'_'+each_element+'_'+self.setup_cannon+'_model.pickle')
                    dataall, metaall, labels_train, offsets, coeffs, covs, scatters, chis, chisqs = pickle.load(use_model)
                    use_model.close()

                    # We will now estimate the line depths based on the model and spectrum flux
                    model_flux     = []
                    spectrum_flux  = []
                    spectrum_error = []

                    for i in range(len(self.cannon['sobject_id'])):

                        features_data = np.hstack((1, params[i] - offsets))
                        newfeatures_data = np.array([np.outer(params[i]-offsets,params[i]-offsets)[np.triu_indices(len(params[i]))]])
                        features_data_final = np.hstack((features_data,newfeatures_data.flatten()))
                        model_gen2 = np.dot(coeffs,features_data_final.T)

                        model_flux.append(model_gen2)
                        spectrum_flux.append(testdataall[:,i,1])
                        spectrum_error.append(testdataall[:,i,2])

                    self.model_flux[each_element] = np.array(model_flux)
                    self.model_chi2[each_element] = chi2each
                    self.spectrum_flux[each_element] = np.array(spectrum_flux)
                    self.spectrum_error[each_element] = np.array(spectrum_error)

                    # Load SNR-depending precision fucntion for X_abund_cannon to increase e_X_abund_cannon
                    precision = np.load(localFilePath+'CANNON/'+self.reduction_DR+'/'+self.version_cannon+'/fits_files/'+self.version_cannon+'_'+self.setup_cannon+'_'+self.reduction_DR+'_'+str(self.nr_ccds_cannon)+'ccds_precision.npy').item()
                    def precision_snr(x, floor, amp, damp):
                        return abs(floor) + amp * np.exp( - x *damp)
                    self.cannon['e_'+each_element+'_abund_cannon'] = (
                            (e_params[:,-1])**2. + 
                            (precision_snr(self.cannon['snr_c2_iraf'], *precision['cannon_precision_function'][each_element]))**2.
                        )**0.5

                    # Estimate which lines are significant detections analogous to SME approach
                    get_detections(self, mode=each_element)

                    # Estimate label distance from 10 closest training set stars for element
                    label_distance(self, mode=each_element)

                except:
                    print('Element '+each_element+' not available!')
                    self.cannon[each_element+'_abund_cannon'][:] = np.nan
                    self.cannon['e_'+each_element+'_abund_cannon'][:] = np.nan
                    self.cannon['flag_'+each_element+'_abund_cannon'][:] = -1
                    self.cannon['ld_'+each_element+'_abund_cannon'][:] = np.nan
                    self.cannon['depth_'+each_element+'_abund_cannon'][:] = np.nan
                    self.cannon['sn_'+each_element+'_abund_cannon'][:] = np.nan
                    self.cannon['chi2_'+each_element+'_abund_cannon'][:] = np.nan

            compute_alpha_fe(self.cannon,sme_cannon='cannon',alpha_elements=['O','Mg','Si','Ca','Ti'])
        else:
            print('You did not call testset.get_SP() and testset.get_IRAF() yet, which initialises the CANNON attribute to save the abundances in!')


    def export_fits(self):
        export_fits(
            self.cannon, 
            filename=localFilePath+'CANNON/'+self.reduction_DR+'/'+self.version_cannon+'/fits_files/'+self.version_cannon+'_'+self.setup_cannon+'_'+self.reduction_DR+'_'+str(self.nr_ccds_cannon)+'ccds_'+str(self.obs_date)
        )

    def plot_HRD(self, savefig=False):
        plot_HRD(self.cannon, sme_cannon='cannon', savefig=savefig)
        
    def plot_alpha(self, savefig=False):
        plot_alpha(self.cannon, sme_cannon='cannon', savefig=savefig)

In [14]:
if sys.argv[1] == '-f':
    print('You are running the Cannon in IPYNB mode')
    version_cannon = 'Cannon3.0.1'
    reduction_DR   = 'dr5.2'
    obs_date       = 131119
    print(version_cannon,reduction_DR,obs_date)
else:
    print('You are running the Cannon in PY mode')
    version_cannon = sys.argv[1]
    reduction_DR   = sys.argv[2]
    obs_date       = sys.argv[3]

wg3_wg4_setup = dict(
    version_reduction  = 'sobject_iraf_52_171009',
    reduction_DR       = reduction_DR,
    version_sme        = 'SME360_DR2_SVN331',
    setup_sme          = 'DR2',
    version_cannon     = version_cannon,
    setup_cannon       = 'SMEmasks_it1',
    nr_ccds_cannon     = 4,
    stellar_parameters = ['Teff', 'Logg', 'Feh', 'Vmic', 'Vsini', 'Ak'],
    elements           = [
                         'Li',  'C',  'O', 'Na', 'Mg', 'Al', 'Si',  'K', 'Ca', 'Sc', 
                         'Ti',  'V', 'Cr', 'Mn', 'Co', 'Ni', 'Cu', 'Zn', 'Rb', 'Sr',
                          'Y', 'Zr', 'Mo', 'Ru', 'Ba', 'La', 'Ce', 'Nd', 'Sm', 'Eu'
                         ]
    )

You are running the Cannon in IPYNB mode
Cannon3.0.1 dr5.2 131119


Now lets create the class for the testset!

In [15]:
if obs_date == 'all':
    obs_date_todo = glob.glob(localFilePath+'CANNON/'+reduction_DR+'/'+version_cannon+'/pickle_test/*Sp*.pickle')
    for each_obs_date in obs_date_todo:
        try:
            each_testset = testset(int(each_obs_date[-18:-12]), wg3_wg4_setup)
            each_testset.get_SP()
            each_testset.get_IRAF()
            each_testset.get_tSNE()
            each_testset.get_AB()
            each_testset.export_fits()
        except:
            pass
else:
    if sys.argv[1] == '-f':
        each_testset = testset(obs_date, wg3_wg4_setup)
    else:
        each_testset = testset(int(obs_date[-13:-7]), wg3_wg4_setup)
    each_testset.get_SP()
    each_testset.get_IRAF()
    each_testset.get_tSNE()
    each_testset.get_AB()
    each_testset.export_fits()
    print('Finished collecting '+str(obs_date))

You are now working in:/shared-storage/buder/svn-repos/trunk/GALAH/CANNON/dr5.2/Cannon3.0.1
pickle_test/Cannon3.0.1_Sp_SMEmasks_it1_dr5.2_4ccds_131119_tags.pickle
Found automatic setup configuration for Li: pickle_test/Cannon3.0.1_Li_SMEmasks_it1_dr5.2_4ccds_131119_tags.pickle
Found automatic setup configuration for C: pickle_test/Cannon3.0.1_C_SMEmasks_it1_dr5.2_4ccds_131119_tags.pickle
Found automatic setup configuration for O: pickle_test/Cannon3.0.1_O_SMEmasks_it1_dr5.2_4ccds_131119_tags.pickle
Found automatic setup configuration for Na: pickle_test/Cannon3.0.1_Na_SMEmasks_it1_dr5.2_4ccds_131119_tags.pickle
Found automatic setup configuration for Mg: pickle_test/Cannon3.0.1_Mg_SMEmasks_it1_dr5.2_4ccds_131119_tags.pickle
Found automatic setup configuration for Al: pickle_test/Cannon3.0.1_Al_SMEmasks_it1_dr5.2_4ccds_131119_tags.pickle
Found automatic setup configuration for Si: pickle_test/Cannon3.0.1_Si_SMEmasks_it1_dr5.2_4ccds_131119_tags.pickle
Found automatic setup configuration 

/home/buder/.local/lib/python2.7/site-packages/ipykernel/__main__.py:21: RuntimeWarning: invalid value encountered in double_scalars


Finished collecting 131119


In [16]:
# Convert IPYNB to PY

os.chdir('/shared-storage/buder/svn-repos/trunk/GALAH/TheGALAHCannon/')

convert_command = 'ipython nbconvert --to script Cannon_collect_test.ipynb'
os.system(convert_command)

os.chdir('/shared-storage/buder/svn-repos/trunk/GALAH/')